# Set beginning of test data

## Description

To get the information required in the .setStartOfCreep methods below, we need to compare the information of loadingSwitch to the information of the real LVDT lines. 

This was done preliminary in Excel, file 20231116-RESUME_SPECIMENS.

It can also be done better here, by using the code in the next section. 

There are three analysis to be made:
* The synchrony between loadingSwitch and the creep system, making sure the valeys/hills of the loadingSwitch coincide with the hills/valleys of the creep system. This allows to consider both time series as synchronous
* For each specimens's series, the time instant that sets the start of creep, which is equal to the point in which the loadingSwitch becomes open (the step in its series)
* Determine how much the data of each specimen must be shifted in time, so all of them coincide at the same time. This may be derived by making sure a notable point coincide in all specimens' series, such as the beginning of creep (identified previously). So, for example, take X minutes before the creep time at each specimen

## Comments related to this specific EXPERIMENT

Esse experimento requeriu uma análise pormenorizada desse algoritmo de "time-stamping" pois notou-se que havia um descompasso entre os pontos de início e fim de creep obtidos aqui, e aqueles obtidos no algoritmo de processamento de dados (processCreepData.ipynb).
O que acontecia era que o ponto de início de creep selecionado com base no presente algoritmo, quando inserido na analise definitiva dos dados, levava a curvas de creep específico que claramente se iniciavam quando a curva de creep ainda estava sob efeito de aumento de carregamento. Em outras palavras, havia um "time-shift" entre as duas análises.

Inicialmente, observou-se que o método utilizado no presente algoritmo para fazer o "time-stamping" dos dados do sistema de aquisição de dados da INEGI utilizava o frequência de amostragem do sistema apenas. Já a análise definitiva utilizava o "time-stamping" feito pelo próprio sistema INEGI e registrado diretamente no arquivo de dados. Como esta última opção parecia ser a mais plausível, pois aparentava levar em conta eventuais oscilações na velocidade de aquisição de dados do sistema, ambos os algoritmos foram compatibilizados para utilizar tal esquema de "time-stamping".

Ao se fazer isso, um novo problema foi observado: agora a curva do interruptor baseado no Arduino não se compatibilizava com os pontos notáveis das curvas do INEGI. Isto é, ao se sincronizar ambos os dados com os impulsos do início das séries de dados, os demais instantes sinalizados como de carregamento completo não se compatibilizavam com o que as curvas dos LVDTs mostravam. Foi-se pensado que isso poderia ser a manifestação de um efeito mecânico de creep intenso devido a alta temperatura, em que se observaria um enorme aumento, atípico, de leitura dos LVDTs mesmo após o carregamento ser completamente transferido. No entanto, as diferentes amostras apresentavam magnitudes de defasamentos entre a curva do interruptor e dos LVDTs distintos entre si. Em outras palavras, na hipótese de se assumir um fenômeno semelhante, não se obtinha concordância plaúsivel entre os dados de diferentes amostras, o que, por sua vez, falava contra a existência de fato de tal fenômeno.

Notou-se que quando se utilizava o "time-stamping" baseado na frequência de amostragem apenas, tal defasamento entre interruptor e sistema da INEGI desaparecia (à excessão do provete SP4, mas este teve um problema relacionado ao interruptor que invalida sua consideração nas análises). Assim, supõe-se que o sistema da INEGI logicmaente possui uma alta estabilidade da amostragem, de forma que se é configurado para amostrar a 20 Hz de fato mantém essa velocidade de amostragem ao longo da sessão, talvez por meio do uso de buffers temporários, e o "time-stamp" inserido no aruqivo de dados considera outros atrasos inerentes ao processo de aquisição de dados, como salvamento dos valores em discos, mas que, por sua vez, não afetam o procesos de aquisição de dados em si. Isso é completamente factível de existir em um sistema de aquisição de dados comercial, e a concordância da tendência dos sinais de LVDT com os sinais do interruptor fala fortemente a favor dessa hipótese, já que estes são dois sistemas eletrônicos totalmente independentes entre si, mas que devem possui comportamento compatibilizado devido à forma mecânica em que estão acoplados.

Assim, conclui-se que a adaptação necessária é fazer o algoritmo "processCreepData.ipynb", e a biblioteca "staticCreepModulus.py" basear seu processo de "time-stamping" não nos valores de tempo do arquivo de dados, mas no valor de frequência de amostragem informado no cabeçalho do arquivo.

## Inspection code necessary to run manually

In [1]:
# Get path of the folder containing the test results
from tkinter.filedialog import askopenfilename, askdirectory
from tkinter import Tk
import os
import csv
import matplotlib.pyplot as plt
import numpy as np

Run the code to read the loadingSwitch data

In [2]:
# Get path of the folder containing the test results
#Read loading switch
window = Tk()
window.wm_attributes('-topmost', 1)
window.withdraw()   # this supress the tk window
loadingSwitchPathFile = askopenfilename(parent=window, title="Open file of loading switch")
Tk().destroy()

loadingSwitchData = np.loadtxt(loadingSwitchPathFile, delimiter=',')
loadingSwitchData[:,1]=(loadingSwitchData[:,1]-loadingSwitchData[0,1])/(1e6) #transform in seconds since beginning
#loadingSwitchData[:,0]=loadingSwitchData[:,0]/max(loadingSwitchData[:,0])

Run the code to read creep system data

Read sampling time information directly from file

In [8]:
from datetime import datetime
from datetime import timedelta
#Inform number of active channels in the system
numberOfChannels=13
#Read first creep test file
window = Tk()
window.wm_attributes('-topmost', 1)
window.withdraw()   # this supress the tk window
creepPathFile = askopenfilename(parent=window, title="Open first file of creep system")
Tk().destroy()

with open(creepPathFile) as f:
    ncols = len(f.readline().split(';'))

creepData = np.loadtxt(creepPathFile, delimiter=';', skiprows=25, usecols=range(1,ncols-2))
#Create time vector
format_string = "%H:%M:%S.%f"
timeVector=[]
samplingInterval=[]
with open(creepPathFile) as file:
    csvreader = csv.reader(file,delimiter=';')
    header = []
    for i in range(0,12+numberOfChannels): header = next(csvreader)
    for iterationRows, row in enumerate(csvreader):
        rawTime=(row[0])
        processedTime = rawTime[:-6] + "." + rawTime[-5:-2]
        if iterationRows == 0:
            timeVector.append(datetime.strptime(processedTime, format_string))
            samplingInterval.append(datetime.strptime(processedTime, format_string))
            lastStrptime=datetime.strptime(processedTime, format_string)
        else:
            timeVector.append((datetime.strptime(processedTime, format_string)-timeVector[0]).total_seconds())
            samplingInterval.append((datetime.strptime(processedTime, format_string)-lastStrptime).total_seconds())
            lastStrptime=datetime.strptime(processedTime, format_string)
timeVector[0]=0
samplingInterval[0]=0.0511
creepData = np.insert(creepData, 0, timeVector, axis=1)
#loadingSwitchData[:,1]=(loadingSwitchData[:,1]-loadingSwitchData[0,1])/(1e6) #transform in seconds since beginning
#loadingSwitchData[:,0]=loadingSwitchData[:,0]-max(loadingSwitchData[:,0])

Approximate sampling time from the sampling frequency

In [3]:
from datetime import datetime
from datetime import timedelta
#Inform number of active channels in the system
numberOfChannels=13
samplingFrequency=100
#Read first creep test file
window = Tk()
window.wm_attributes('-topmost', 1)
window.withdraw()   # this supress the tk window
creepPathFile = askopenfilename(parent=window, title="Open first file of creep system")
Tk().destroy()

# with open(creepPathFile) as f:
#     ncols = len(f.readline().split(';'))

ncols = numberOfChannels

creepData2 = np.loadtxt(creepPathFile, delimiter=';', skiprows=25, usecols=range(1,ncols+1))
#Create time vector
format_string = "%H:%M:%S.%f"
timeVector2=[]
with open(creepPathFile) as file:
    csvreader = csv.reader(file,delimiter=';')
    header = []
    for i in range(0,12+numberOfChannels): header = next(csvreader)
    for iterationRows, row in enumerate(csvreader):
        if iterationRows == 0:
            timeVector2.append(1/samplingFrequency)
        else:
            timeVector2.append(timeVector2[iterationRows-1]+1/samplingFrequency)
timeVector2[0]=0
creepData2 = np.insert(creepData2, 0, timeVector2, axis=1)
#loadingSwitchData[:,1]=(loadingSwitchData[:,1]-loadingSwitchData[0,1])/(1e6) #transform in seconds since beginning
#loadingSwitchData[:,0]=loadingSwitchData[:,0]-max(loadingSwitchData[:,0])

Compare both time-stamping methods

In [55]:
%matplotlib qt
#loadingSwitchOffset=18.637+0.416-0.830+1.393+0.31+0.20-130.58
#loadingSwitchOffset=0
#loadingSwitchOffset=18.637+0.416-0.830+1.393+0.31+0.20+22+1.206-132
#plt.plot(timeVector2,creepData2[:,8]/max(creepData2[:,8]), label="Approx")
#plt.plot(timeVector,creepData[:,8]/max(creepData[:,8]), label="Exact")
#plt.plot([(value1-value2) for value1,value2 in zip(timeVector,timeVector2) if value1!=0], label="difference between exact and approx.")
#plt.plot([value-1/20 for value in samplingInterval], label="relation between sampling intervals exact and approx.")
plt.scatter(np.arange(0, len(samplingInterval)),samplingInterval, label="relation between sampling intervals exact and approx.")
#plt.plot([(value2-value1)/value1 for value1,value2 in zip(timeVector,timeVector2) if value1!=0], label="difference between exact and approx.")
#plt.plot([value1/value2 for value1,value2 in zip(timeVector,timeVector2) if value2!=0], label="difference between exact and approx.")
plt.legend()
#plt.xlim([0,80])
plt.show()

Plot switch and LVDT to compatibilize their histories

In [4]:
%matplotlib qt
#loadingSwitchOffset=18.637+0.416-0.830+1.393+0.31+0.20-130.58
#loadingSwitchOffset=0
loadingSwitchOffset=-29.41+1.509-9.11
#loadingSwitchOffset=18.637+0.416-0.830+1.393+0.31+0.20+22+1.206-132
plt.plot(loadingSwitchData[:,1]+loadingSwitchOffset,loadingSwitchData[:,0]/max(loadingSwitchData[:,0]), label="Loading switch")
plt.plot(timeVector2,creepData2[:,3]/max(creepData2[:,3]), label="Creep system")
plt.legend()
#plt.xlim([0,80])
plt.show()

Now that loadingSwitchOffset is done, we can inspect for each specimen by selecting the proper lines below and plotting them

In [7]:
creepData=creepData2
timeVector=timeVector2

In [8]:
#SP2
%matplotlib qt
fig, ax1 = plt.subplots()
ax1.plot(loadingSwitchData[:,1]+loadingSwitchOffset,loadingSwitchData[:,0]/max(loadingSwitchData[:,0]),linestyle="--", color='r', label="Loading switch")
ax1.plot(creepData[:,0],creepData[:,13]/max(creepData[:,13]),linestyle="--", color='k', label="Load cell")

ax2=ax1.twinx()
ax2.plot(creepData[:,0],creepData[:,6], label="SP_1-LVDT_2A")
ax2.plot(creepData[:,0],creepData[:,5], label="SP_1-LVDT_1A")
ax2.plot(creepData[:,0],creepData[:,8], label="SP_1-LVDT_4A")

ax2.plot(creepData[:,0],creepData[:,10], label="SP_2-LVDT_5A")
ax2.plot(creepData[:,0],creepData[:,9], label="SP_2-LVDT_05")
ax2.plot(creepData[:,0],creepData[:,7], label="SP_2-LVDT-3A")

ax2.plot(creepData[:,0],creepData[:,11], label="SP_3-LVDT_07")
ax2.plot(creepData[:,0],creepData[:,12], label="SP_3-LVDT_09")
ax2.plot(creepData[:,0],creepData[:,4], label="SP_3-LVDT_10")

ax2.plot(creepData[:,0],creepData[:,2], label="SP_4-LVDT_00")
ax2.plot(creepData[:,0],creepData[:,3], label="SP_4-LVDT_01")
ax2.plot(creepData[:,0],creepData[:,1], label="SP_4-LVDT_HS25")

fig.legend()
#plt.xlim([0,80])
plt.show()

##### Results:
#SP2 full load starts 219.734 seconds after test start 

In [68]:
#SP2
%matplotlib qt
fig, ax1 = plt.subplots()
ax1.plot(loadingSwitchData[:,1]+loadingSwitchOffset,loadingSwitchData[:,0]/max(loadingSwitchData[:,0]),linestyle="--", color='r', label="Loading switch")
ax2=ax1.twinx()
ax2.plot(creepData[:,0],creepData[:,8], label="SP_2-LVDT_1A")
ax2.plot(creepData[:,0],creepData[:,9], label="SP_2-LVDT_2A")
ax2.plot(creepData[:,0],creepData[:,11], label="SP_2-LVDT_4A")
fig.legend()
#plt.xlim([0,80])
plt.show()

##### Results:
#SP2 full load starts 219.734 seconds after test start 

In [63]:
#SP4
%matplotlib qt
fig, ax1 = plt.subplots()
ax1.plot(loadingSwitchData[:,1]+loadingSwitchOffset,loadingSwitchData[:,0]/max(loadingSwitchData[:,0]),linestyle="--", color='r', label="Loading switch")
ax2=ax1.twinx()
ax2.plot(creepData[:,0],creepData[:,10], label="SP_4-LVDT_3A")
ax2.plot(creepData[:,0],creepData[:,4], label="SP_4-LVDT_5")
ax2.plot(creepData[:,0],creepData[:,12], label="SP_4-LVDT-5A")
fig.legend()
#plt.xlim([0,80])
plt.show()

##### Results:
#SP4 full load starts 413.758 seconds after test start 

In [62]:
#SP3
%matplotlib qt
fig, ax1 = plt.subplots()
ax1.plot(loadingSwitchData[:,1]+loadingSwitchOffset,loadingSwitchData[:,0]/max(loadingSwitchData[:,0]),linestyle="--", color='r', label="Loading switch")
ax2=ax1.twinx()
ax2.plot(creepData[:,0],creepData[:,5], label="SP_3-LVDT_07")
ax2.plot(creepData[:,0],creepData[:,6], label="SP_3-LVDT_09")
ax2.plot(creepData[:,0],creepData[:,7], label="SP_3-LVDT_10")
fig.legend()
#plt.xlim([0,80])
plt.show()

##### Results:
#SP3 full load starts 859.5356 seconds after test start 

In [60]:
#SP5
%matplotlib qt
fig, ax1 = plt.subplots()
ax1.plot(loadingSwitchData[:,1]+loadingSwitchOffset,loadingSwitchData[:,0]/max(loadingSwitchData[:,0]),linestyle="--", color='r', label="Loading switch")
ax2=ax1.twinx()
ax2.plot(creepData[:,0],creepData[:,2], label="SP_5-LVDT_00")
ax2.plot(creepData[:,0],creepData[:,3], label="SP_5-LVDT_01")
ax2.plot(creepData[:,0],creepData[:,1], label="SP_5-LVDT_HS10")
fig.legend() 
#plt.xlim([0,80])
plt.show()

##### Results:
#SP5 full load starts 1011.763 seconds after test start 